In [1]:
import syft as sy
sy.__version__

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/ollytvyn/Projects/01_private_ai/PySyft


'0.8.0-beta.6'

In [2]:
import inspect
from inspect import _signature_fromstr
import types
import re
from types import FunctionType, LambdaType, ModuleType
from typing import Callable

import numpy as np

# func_signature = signature(np.array) # no signature found for builtin <built-in function array>

# func_signature = signature(np.all) # numpy._globals._NoValueType not in TYPE_BANK

func_signature = inspect.signature(np.zeros_like)
display(func_signature)
func_signature2 = inspect.signature(np.all)
display(func_signature2)
# sy.serialize(func_signature2)

<Signature (a, dtype=None, order='K', subok=True, shape=None)>

<Signature (a, axis=None, out=None, keepdims=<no value>, *, where=<no value>)>

In [5]:
function_signatures_registry = {
    "concatenate": "concatenate(a1,a2, *args,axis=0,out=None,dtype=None,casting='same_kind')",
    "set_numeric_ops": "set_numeric_ops(op1,op2, *args)",
    "geterrorobj": "geterrobj()",
    "source": "source(object, output)",
    "fromstring": "fromstring(string, dtype=None, count=-1, like=None)"
}

def get_signature_from_docstring(doc: str, callable_name: str) -> str:
    if not doc or callable_name not in doc:
        return None
    else:
        doc = re.sub(r'\s',"", doc.split('\n\n')[0])
        search_res = re.search(rf"{callable_name}\((.+)\)", doc)
        if search_res:
            signature = search_res.group(1)
            # decomposing "[]" optional  params
            params = re.findall(r"\[(.+?)\]", signature)
            if params:
                print(callable_name)
                for param in params[:-1]:
                    signature = signature.replace(f"[{param}]", param)
            
                if re.search(fr"(?<={params[-1]})\],", signature):
                    signature = signature.replace(f"[{params[-1]}],", params[-1])
                else:
                    signature = signature.replace(f"[{params[-1]}]",  f', {",".join([f"{param}=None" for param in params[-1].split(",") if param])}')

            signature = re.sub(r",(\/|\*)", "", signature)
            signature = re.sub(r"dtype=(\w+),", "dtype=None,", signature)
            return f"{callable_name}({signature})"
        else:
            return None

def get_signature_from_registry(callable_name: str) -> str:
    return function_signatures_registry[callable_name]


def generate_signature(_callable) -> inspect.Signature:
    name, doc = _callable.__name__, _callable.__doc__
    # returning predefined signature if in signature registry
    name_in_registry = name in function_signatures_registry.keys()
    text_signature = get_signature_from_registry(name) \
        if name_in_registry else  get_signature_from_docstring(doc, name)
    # TODO safe handling if function signature can not be generated
    text_signature = "()" if text_signature is None else  text_signature  
    return _signature_fromstr(inspect.Signature, _callable, text_signature, True)


def generate_signature_2(_callable):
    pass


def get_signature(_callable) -> inspect.Signature:
    try:
        res = inspect.signature(_callable)
        if res is None or "OutStream" in str(res):
            raise ValueError("")
        else:
            return res
    except Exception:
        return generate_signature(_callable)

In [6]:



for lib_module in [np]:
    parent_path = lib_module.__name__
    for attr in dir(lib_module):
        lib_obj = getattr(lib_module, attr)
        path = f"{parent_path}.{attr}"  
        if inspect.isfunction(lib_obj) or (callable(lib_obj) and not inspect.isclass(lib_obj)):
            signature = get_signature(lib_obj)
            try:
                sy.serialize(signature)
            except Exception as e:
                print(f"skipping {lib_obj} with signature: {signature}")
                pass

absolute
absolute
add
arange
arccos
arccosh
arcsin
arcsinh
arctan
arctan2
arctanh
bitwise_and
invert
bitwise_or
bitwise_xor
cbrt
ceil
conjugate
conjugate
copysign
cos
cosh
deg2rad
degrees
divide
divmod
equal
exp
exp2
expm1
fabs
float_power
floor
floor_divide
fmax
fmin
fmod
frexp
frompyfunc
gcd
greater
greater_equal
heaviside
hypot
invert
isfinite
isinf
isnan
isnat
lcm
ldexp
left_shift
less
less_equal
log
log10
log1p
log2
logaddexp
logaddexp2
logical_and
logical_not
logical_or
logical_xor
matmul
maximum
minimum
remainder
modf
multiply
negative
nextafter
not_equal
positive
power
rad2deg
radians
reciprocal
remainder
right_shift
rint
sign
signbit
sin
sinh
spacing
sqrt
square
subtract
tan
tanh
divide
trunc
where


In [ ]:
def get_signature_from_docstring(doc: str, callable_name: str) -> str:
    if not doc or callable_name not in doc:
        return None
    else:
        doc = re.sub(r'\s',"", doc.split('\n\n')[0])
        search_res = re.search(rf"{callable_name}\((.+)\)", doc)
        if search_res:
            signature = search_res.group(1)
            # decomposing "[]" optional  params
            params = re.findall(r"\[(.+?)\]", signature)
            if params:
                for param in params[:-1]:
                    signature = signature.replace(f"[{param}]", param)
            
                if re.search(fr"(?<={params[-1]})\],", signature):
                    signature = signature.replace(f"[{params[-1]}],", params[-1])
                else:
                    signature = signature.replace(f"[{params[-1]}]",  f', {",".join([f"{param}=None" for param in params[-1].split(",") if param])}')

            signature = re.sub(r",(\/|\*)", "", signature)
            signature = re.sub(r"dtype=(\w+),", "dtype=None,", signature)
            return f"{callable_name}({signature})"
        else:
            return None

_callable = np.test        
sig = get_signature_from_docstring(_callable.__doc__, _callable.__name__)
sig

In [ ]:
display(_callable)
signature = get_signature(_callable)
display(signature)

sy.serialize(signature)

In [21]:
# inspect.signature(np.test)
inspect.signature(np.source)

<Signature (object, output=<ipykernel.iostream.OutStream object at 0x103958640>)>

In [16]:
# np._pytesttester.PytestTester
check_funciton = lambda func: func if (inspect.isfunction(getattr(np, func)) or ((callable(getattr(np, func))) and not inspect.isclass(getattr(np, func)))) else None
[x for x in list(map(check_funciton, dir(np))) if x is not None]
# inspect.isfunction(np.all)

['__dir__',
 '__getattr__',
 '_add_newdoc_ufunc',
 '_get_promotion_state',
 '_no_nep50_warning',
 '_pyinstaller_hooks_dir',
 '_set_promotion_state',
 'abs',
 'absolute',
 'add',
 'add_docstring',
 'add_newdoc',
 'add_newdoc_ufunc',
 'all',
 'allclose',
 'alltrue',
 'amax',
 'amin',
 'angle',
 'any',
 'append',
 'apply_along_axis',
 'apply_over_axes',
 'arange',
 'arccos',
 'arccosh',
 'arcsin',
 'arcsinh',
 'arctan',
 'arctan2',
 'arctanh',
 'argmax',
 'argmin',
 'argpartition',
 'argsort',
 'argwhere',
 'around',
 'array',
 'array2string',
 'array_equal',
 'array_equiv',
 'array_repr',
 'array_split',
 'array_str',
 'asanyarray',
 'asarray',
 'asarray_chkfinite',
 'ascontiguousarray',
 'asfarray',
 'asfortranarray',
 'asmatrix',
 'atleast_1d',
 'atleast_2d',
 'atleast_3d',
 'average',
 'bartlett',
 'base_repr',
 'binary_repr',
 'bincount',
 'bitwise_and',
 'bitwise_not',
 'bitwise_or',
 'bitwise_xor',
 'blackman',
 'block',
 'bmat',
 'broadcast_arrays',
 'broadcast_shapes',
 'broadcast_to',
 'busday_count',
 'busday_offset',
 'byte_bounds',
 'can_cast',
 'cbrt',
 'ceil',
 'choose',
 'clip',
 'column_stack',
 'common_type',
 'compare_chararrays',
 'compress',
 'concatenate',
 'conj',
 'conjugate',
 'convolve',
 'copy',
 'copysign',
 'copyto',
 'corrcoef',
 'correlate',
 'cos',
 'cosh',
 'count_nonzero',
 'cov',
 'cross',
 'cumprod',
 'cumproduct',
 'cumsum',
 'datetime_as_string',
 'datetime_data',
 'deg2rad',
 'degrees',
 'delete',
 'deprecate',
 'deprecate_with_doc',
 'diag',
 'diag_indices',
 'diag_indices_from',
 'diagflat',
 'diagonal',
 'diff',
 'digitize',
 'disp',
 'divide',
 'divmod',
 'dot',
 'dsplit',
 'dstack',
 'ediff1d',
 'einsum',
 'einsum_path',
 'empty',
 'empty_like',
 'equal',
 'exp',
 'exp2',
 'expand_dims',
 'expm1',
 'extract',
 'eye',
 'fabs',
 'fastCopyAndTranspose',
 'fill_diagonal',
 'find_common_type',
 'fix',
 'flatnonzero',
 'flip',
 'fliplr',
 'flipud',
 'float_power',
 'floor',
 'floor_divide',
 'fmax',
 'fmin',
 'fmod',
 'format_float_positional',
 'format_float_scientific',
 'frexp',
 'from_dlpack',
 'frombuffer',
 'fromfile',
 'fromfunction',
 'fromiter',
 'frompyfunc',
 'fromregex',
 'fromstring',
 'full',
 'full_like',
 'gcd',
 'genfromtxt',
 'geomspace',
 'get_array_wrap',
 'get_include',
 'get_printoptions',
 'getbufsize',
 'geterr',
 'geterrcall',
 'geterrobj',
 'gradient',
 'greater',
 'greater_equal',
 'hamming',
 'hanning',
 'heaviside',
 'histogram',
 'histogram2d',
 'histogram_bin_edges',
 'histogramdd',
 'hsplit',
 'hstack',
 'hypot',
 'i0',
 'identity',
 'imag',
 'in1d',
 'indices',
 'info',
 'inner',
 'insert',
 'interp',
 'intersect1d',
 'invert',
 'is_busday',
 'isclose',
 'iscomplex',
 'iscomplexobj',
 'isfinite',
 'isfortran',
 'isin',
 'isinf',
 'isnan',
 'isnat',
 'isneginf',
 'isposinf',
 'isreal',
 'isrealobj',
 'isscalar',
 'issctype',
 'issubclass_',
 'issubdtype',
 'issubsctype',
 'iterable',
 'ix_',
 'kaiser',
 'kron',
 'lcm',
 'ldexp',
 'left_shift',
 'less',
 'less_equal',
 'lexsort',
 'linspace',
 'load',
 'loadtxt',
 'log',
 'log10',
 'log1p',
 'log2',
 'logaddexp',
 'logaddexp2',
 'logical_and',
 'logical_not',
 'logical_or',
 'logical_xor',
 'logspace',
 'lookfor',
 'mask_indices',
 'mat',
 'matmul',
 'max',
 'maximum',
 'maximum_sctype',
 'may_share_memory',
 'mean',
 'median',
 'meshgrid',
 'min',
 'min_scalar_type',
 'minimum',
 'mintypecode',
 'mod',
 'modf',
 'moveaxis',
 'msort',
 'multiply',
 'nan_to_num',
 'nanargmax',
 'nanargmin',
 'nancumprod',
 'nancumsum',
 'nanmax',
 'nanmean',
 'nanmedian',
 'nanmin',
 'nanpercentile',
 'nanprod',
 'nanquantile',
 'nanstd',
 'nansum',
 'nanvar',
 'ndim',
 'negative',
 'nested_iters',
 'nextafter',
 'nonzero',
 'not_equal',
 'obj2sctype',
 'ones',
 'ones_like',
 'outer',
 'packbits',
 'pad',
 'partition',
 'percentile',
 'piecewise',
 'place',
 'poly',
 'polyadd',
 'polyder',
 'polydiv',
 'polyfit',
 'polyint',
 'polymul',
 'polysub',
 'polyval',
 'positive',
 'power',
 'printoptions',
 'prod',
 'product',
 'promote

In [34]:
_callable = np.fromstring
inspect._signature_fromstr(inspect.Signature, _callable, "fromstring(string, dtype=None, count=-1, like=None)")

RuntimeError: 

In [ ]:
sig = "string, dtype=None, count=-1, sep, like=None"
optional = False
params = sig.split(', ')
for idx in range(len(params)):
    match = re.match(r"(\w+)=(\S+)", params[idx])
    if not optional and match:
        optional=True
    elif optional and not match:
        params[idx]=f"{params[idx]}=None"
    else:
        continue
    
        

params   

In [ ]:
inspect.signature(np.zeros_like)

In [ ]:
", ".join([f"{param}=None" for param in ", signature, extobj".split(", ") if param])

In [ ]:
sig_str = get_text_signature(np.concatenate.__doc__)


inspect._signature_fromstr(inspect.Signature, np.arange, "()", True)

In [ ]:
# inspect.signature(inspect.Parameter.replace)
inspect.signature(np.concatenate).parameters

In [ ]:
custo_arrange = np.arange
custo_arrange.__signature__ = inspect._signature_fromstr(inspect.Signature, np.arange, "arange(start, stop, step, dtype=None, like=None)", True)
